In [1]:
# 导入各种图表
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line, Pie
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

#加载数据
data = pd.read_excel("data/table_data_batch_1_to_100.xlsx")

In [13]:
# 机具品目按照购买数量(台)绘制饼图
item = data.groupby('机具品目').agg({'购买数量(台)': 'sum'}).sort_values(by='购买数量(台)', ascending=False)[:10].to_dict()['购买数量(台)']
# | 占位符     | 含义          | 示例     |
# | ------- | ----------- | ------ |
# | **{b}** | 数据项名称（name） | “郑州市”  |
# | **{c}** | 数据项值（value） | “1234” |
# | **{d}** | 百分比（自动计算）   | “12.5” |

c = (
    Pie()
    .add("机具品目", [*item.items()])
    .set_series_opts(label_opts=opts.LabelOpts(font_size=16, formatter="{b}: {c} ({d}%)"))
    .set_global_opts(
    legend_opts=opts.LegendOpts(
        type_="scroll",
        orient="horizontal",
        pos_top="1%",
        item_gap=20,
        textstyle_opts=opts.TextStyleOpts(font_size=20),  # 图例字体大小
    )
)
)
c.render_notebook()

In [14]:
#
from pyecharts.charts import Bar
from pyecharts import options as opts

# ======= 数据处理部分（与你原来一致）=======
county_stats = data.groupby('县').agg({
    '单台中央补贴额(元)': ['sum', 'mean', 'count']
}).round(2)

county_stats.columns = ['总补贴金额', '平均补贴金额', '设备数量']
county_stats = county_stats.sort_values('总补贴金额', ascending=True)

counties = county_stats.index.tolist()
values = county_stats['总补贴金额'].tolist()

# ======= pyecharts 条形图 =======
bar = (
    Bar()
    .add_xaxis(counties)     # x 轴为县名称（横向图中是 y 轴方向）
    .add_yaxis(
        "总补贴金额",
        values,
        label_opts=opts.LabelOpts(
            position="right",          # 值显示在右边
            font_size=12,              # 标签字体
            formatter="{c}"            # 只显示数值
        )
    )
    .reversal_axis()               # 反转 → 变成水平条形图
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="各县补贴金额总和",
            title_textstyle_opts=opts.TextStyleOpts(font_size=18),
            pos_top="1%"
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=12)
        ),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=12)
        ),
        legend_opts=opts.LegendOpts(
            textstyle_opts=opts.TextStyleOpts(font_size=14),
            item_width=20,
            item_height=12
        )
    )
)

# 输出图表
bar.render_notebook()

In [15]:
#月度补贴趋势
from pyecharts.charts import Line
from pyecharts import options as opts

data['购机日期'] = pd.to_datetime(data['购机日期'])

# 按月统计
monthly_stats = data.groupby(data['购机日期'].dt.to_period('M')).agg({
    '单台中央补贴额(元)': ['sum', 'count']
})
monthly_stats.columns = ['月度补贴金额', '月度购置数量']

months = monthly_stats.index.astype(str).tolist()
money_values = monthly_stats['月度补贴金额'].tolist()

# ===== pyecharts 折线图 =====
line = (
    Line()
    .add_xaxis(months)
    .add_yaxis(
        "月度补贴金额",
        money_values,
        is_smooth=False,
        symbol="circle",              # 数据点圆形
        symbol_size=8,
        label_opts=opts.LabelOpts(is_show=True, font_size=12),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="月度补贴金额趋势",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20),
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=45, font_size=12)
        ),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=12)
        ),
        legend_opts=opts.LegendOpts(
            textstyle_opts=opts.TextStyleOpts(font_size=14)
        ),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[opts.DataZoomOpts()],  # 可缩放
    )
)

line.render_notebook()

In [16]:
#月度购置数量
from pyecharts.charts import Bar

count_values = monthly_stats['月度购置数量'].tolist()

# ===== pyecharts 柱状图 =====
bar = (
    Bar()
    .add_xaxis(months)
    .add_yaxis(
        "月度购置数量",
        count_values,
        label_opts=opts.LabelOpts(is_show=True, font_size=12)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="月度设备购置数量",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20),
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=45, font_size=12)
        ),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=12)
        ),
        legend_opts=opts.LegendOpts(
            textstyle_opts=opts.TextStyleOpts(font_size=14)
        ),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[opts.DataZoomOpts()],
    )
)

bar.render_notebook()

In [17]:
#生产厂家分布 top20
from pyecharts.charts import Bar
from pyecharts import options as opts

# 数据
manufacturer_counts = data['生产厂家'].value_counts().head(20)

x_data = manufacturer_counts.index.tolist()
y_data = manufacturer_counts.tolist()

bar1 = (
    Bar()
    .add_xaxis(x_data)
    .add_yaxis(
        "设备数量（台）",
        y_data,
        label_opts=opts.LabelOpts(is_show=True, position="top", font_size=12)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="主要生产厂家分布（Top 20）",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=45, font_size=12)
        ),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=12)
        ),
        legend_opts=opts.LegendOpts(
            textstyle_opts=opts.TextStyleOpts(font_size=14)
        ),
        datazoom_opts=[opts.DataZoomOpts()],   # 可缩放
        tooltip_opts=opts.TooltipOpts(trigger="axis")
    )
)

bar1.render_notebook()

In [20]:
#生产厂家补贴金额_top20
manufacturer_subsidy = (
    data.groupby('生产厂家')['单台中央补贴额(元)']
    .sum()
    .sort_values(ascending=False)
    .head(20)
)

x2 = manufacturer_subsidy.index.tolist()
y2 = manufacturer_subsidy.tolist()

bar2 = (
    Bar()
    .add_xaxis(x2)
    .add_yaxis(
        "补贴金额（元）",
        y2,
        label_opts=opts.LabelOpts(
            is_show=True,
            position="top",
            font_size=12,
            formatter="{c}"     # 显示数值
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="主要生产厂家补贴金额（Top 20）",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=45, font_size=12)
        ),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=12)
        ),
        legend_opts=opts.LegendOpts(
            textstyle_opts=opts.TextStyleOpts(font_size=14)
        ),

        datazoom_opts=[opts.DataZoomOpts()],
        tooltip_opts=opts.TooltipOpts(trigger="axis")
    )
)

bar2.render_notebook()

In [1]:
from pyecharts import options as opts
from pyecharts.charts import Map
import pandas as pd
data = pd.read_excel("data/table_data_all.xlsx")
# data = pd.read_excel("data/table_data_batch_1_to_100.xlsx")

print(data["县"].unique())

mapping = {
        # 郑州市
        '中原区': '郑州市', '二七区': '郑州市', '管城回族区': '郑州市', '金水区': '郑州市',
        '上街区': '郑州市', '惠济区': '郑州市', '中牟县': '郑州市', '巩义市': '郑州市',
        '荥阳市': '郑州市', '新密市': '郑州市', '新郑市': '郑州市', '登封市': '郑州市',
        '郑东新区': '郑州市', '高新区': '郑州市', '经开区': '郑州市', '航空港经济试验区': '郑州市',

        # 开封市
        '龙亭区': '开封市', '顺河回族区': '开封市', '鼓楼区': '开封市', '禹王台区': '开封市',
        '祥符区': '开封市', '杞县': '开封市', '通许县': '开封市', '尉氏县': '开封市',
        '兰考县': '开封市', '示范区': '开封市', '市示范区': '开封市',

        # 洛阳市
        '老城区': '洛阳市', '西工区': '洛阳市', '瀍河回族区': '洛阳市', '涧西区': '洛阳市',
        '吉利区': '洛阳市', '洛龙区': '洛阳市', '孟津县': '洛阳市', '新安县': '洛阳市',
        '栾川县': '洛阳市', '嵩县': '洛阳市', '汝阳县': '洛阳市', '宜阳县': '洛阳市',
        '洛宁县': '洛阳市', '伊川县': '洛阳市', '偃师市': '洛阳市', '伊滨区': '洛阳市',

        # 平顶山市
        '新华区': '平顶山市', '卫东区': '平顶山市', '石龙区': '平顶山市', '湛河区': '平顶山市',
        '宝丰县': '平顶山市', '叶县': '平顶山市', '鲁山县': '平顶山市', '郏县': '平顶山市',
        '舞钢市': '平顶山市', '汝州市': '平顶山市',

        # 安阳市
        '文峰区': '安阳市', '北关区': '安阳市', '殷都区': '安阳市', '龙安区': '安阳市',
        '安阳县': '安阳市', '汤阴县': '安阳市', '滑县': '安阳市', '内黄县': '安阳市',
        '林州市': '安阳市',

        # 鹤壁市
        '鹤山区': '鹤壁市', '山城区': '鹤壁市', '淇滨区': '鹤壁市', '浚县': '鹤壁市',
        '淇县': '鹤壁市',

        # 新乡市
        '红旗区': '新乡市', '卫滨区': '新乡市', '凤泉区': '新乡市', '牧野区': '新乡市',
        '新乡县': '新乡市', '获嘉县': '新乡市', '原阳县': '新乡市', '延津县': '新乡市',
        '封丘县': '新乡市', '长垣县': '新乡市', '卫辉市': '新乡市', '辉县市': '新乡市',
        '平原示范区': '新乡市',

        # 焦作市
        '解放区': '焦作市', '中站区': '焦作市', '马村区': '焦作市', '山阳区': '焦作市',
        '修武县': '焦作市', '博爱县': '焦作市', '武陟县': '焦作市', '温县': '焦作市',
        '沁阳市': '焦作市', '孟州市': '焦作市',

        # 濮阳市
        '华龙区': '濮阳市', '清丰县': '濮阳市', '南乐县': '濮阳市', '范县': '濮阳市',
        '台前县': '濮阳市', '濮阳县': '濮阳市',

        # 许昌市
        '魏都区': '许昌市', '建安区': '许昌市', '鄢陵县': '许昌市', '襄城县': '许昌市',
        '禹州市': '许昌市', '长葛市': '许昌市',

        # 漯河市
        '源汇区': '漯河市', '郾城区': '漯河市', '召陵区': '漯河市', '舞阳县': '漯河市',
        '临颍县': '漯河市',

        # 三门峡市
        '湖滨区': '三门峡市', '陕州区': '三门峡市', '渑池县': '三门峡市', '卢氏县': '三门峡市',
        '义马市': '三门峡市', '灵宝市': '三门峡市',

        # 南阳市
        '宛城区': '南阳市', '卧龙区': '南阳市', '南召县': '南阳市', '方城县': '南阳市',
        '西峡县': '南阳市', '镇平县': '南阳市', '内乡县': '南阳市', '淅川县': '南阳市',
        '社旗县': '南阳市', '唐河县': '南阳市', '新野县': '南阳市', '桐柏县': '南阳市',
        '邓州市': '南阳市',

        # 商丘市
        '梁园区': '商丘市', '睢阳区': '商丘市', '民权县': '商丘市', '睢县': '商丘市',
        '宁陵县': '商丘市', '柘城县': '商丘市', '虞城县': '商丘市', '夏邑县': '商丘市',
        '永城市': '商丘市',

        # 信阳市
        '浉河区': '信阳市', '平桥区': '信阳市', '罗山县': '信阳市', '光山县': '信阳市',
        '新县': '信阳市', '商城县': '信阳市', '固始县': '信阳市', '潢川县': '信阳市',
        '淮滨县': '信阳市', '息县': '信阳市',

        # 周口市
        '川汇区': '周口市', '扶沟县': '周口市', '西华县': '周口市', '商水县': '周口市',
        '沈丘县': '周口市', '郸城县': '周口市', '淮阳县': '周口市', '太康县': '周口市',
        '鹿邑县': '周口市', '项城市': '周口市', '黄泛区': '周口市',

        # 驻马店市
        '驿城区': '驻马店市', '西平县': '驻马店市', '上蔡县': '驻马店市', '平舆县': '驻马店市',
        '正阳县': '驻马店市', '确山县': '驻马店市', '泌阳县': '驻马店市', '汝南县': '驻马店市',
        '遂平县': '驻马店市', '新蔡县': '驻马店市',

        # 济源市
        '济源': '济源市', '济源市': '济源市',

        # 特殊功能区（需要根据实际情况调整）
        '示范区': '开封市',  # 默认分配到开封市，需要根据实际情况调整
        '开发区': '郑州市',   # 默认分配到郑州市，需要根据实际情况调整
        '工业园区': '郑州市', # 默认分配到郑州市，需要根据实际情况调整
        '经济技术开发区': '郑州市', # 默认分配到郑州市
        '新城区': '郑州市',   # 默认分配到郑州市
        '城乡一体化示范区': '郑州市', # 默认分配到郑州市
        '东城区': '郑州市',   # 默认分配到郑州市
    }
data['县'] = data['县'].map(lambda x: mapping.get(x.strip(), x.strip()))
print("--------------")
print(data["县"].unique())

result2 = data.groupby('县').agg({'购买数量(台)':'count'}).to_dict()['购买数量(台)']
map_chart = (
    Map()
    .add("购买数量", [*result2.items()],  maptype="河南",is_map_symbol_show=False)  # 可替换为具体省市 GeoJSON
    .set_global_opts(
        title_opts=opts.TitleOpts(title="县级农机分布地图"),
        visualmap_opts=opts.VisualMapOpts(max_=max(result2.values()))
    )
)
map_chart.render_notebook()

['淮阳县' '宝丰县' '延津县' '荥阳市' '示范区' '鲁山县' '山城区' '项城市' '驿城区' '西平县' '陕州区' '郸城县'
 '鼓楼区' '市示范区' '林州市' '太康县' '唐河县' '嵩县' '孟津县' '伊川县' '开发区' '博爱县' '淮滨县' '沁阳市'
 '遂平县' '经开区' '修武县' '鹿邑县' '新安县' '殷都区' '杞县' '西华县' '宜阳县' '汝州市' '禹州市' '睢县'
 '栾川县' '桐柏县' '平桥区' '登封市' '淇滨区' '正阳县' '叶县' '龙安区' '舞钢市' '扶沟县' '原阳县' '鹤山区'
 '社旗县' '确山县' '获嘉县' '宁陵县' '新蔡县' '黄泛区' '柘城县' '睢阳区' '濮阳县' '召陵区' '沈丘县'
 '航空港经济试验区' '川汇区' '平原示范区' '潢川县' '渑池县' '北关区' '新乡县' '辉县市' '汤阴县' '武陟县' '虞城县'
 '南乐县' '范县' '息县' '城乡一体化示范区' '洛龙区' '台前县' '新郑市' '泌阳县' '滑县' '封丘县' '方城县' '偃师市'
 '淇县' '卫辉市' '浚县' '新野县' '源汇区' '内黄县' '济源' '惠济区' '洛宁县' '民权县' '上蔡县' '魏都区'
 '通许县' '安阳县' '建安区' '文峰区' '中牟县' '巩义市' '兰考县' '凤泉区' '夏邑县' '长葛市' '平舆县' '义马市'
 '郏县' '牧野区' '西峡县' '镇平县' '梁园区' '固始县' '华龙区' '湖滨区' '灵宝市' '商城县' '上街区' '龙亭区'
 '伊滨区' '马村区' '汝南县' '高新区' '卫滨区' '光山县' '清丰县' '东城区' '卢氏县' '新密市' '尉氏县' '鄢陵县'
 '永城市' '顺河回族区' '长垣县' '浉河区' '新县' '舞阳县' '淅川县' '南召县' '郑东新区' '禹王台区' '商水县'
 '祥符区' '工业园区' '经济技术开发区' '新城区' '温县' '汝阳县' '新华区' '湛河区' '内乡县' '邓州市' '襄城县'
 '卫东区' '卧龙区' '罗山县' '郾城区' '临颍县' '宛城区' '石龙区' '孟州市']
--------------
[